In [1]:
import json
import io
import os
from collections import Counter

# Imports the Google Cloud client library
from google.cloud import vision

In [2]:
# Opening JSON file
f = open('insta_post1.json')

In [3]:
# returns JSON object as 
# a dictionary
data = json.load(f)

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/kinnarypanchal/Downloads/uda-assignment-4-96341864fcec.json"

In [5]:
def detect_labels_uri(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations
    
    lbs = list()
    scr = list()
    
#     return labels
    
#     print('Labels:')


    for label in labels:
        lbs.append(label.description)
        scr.append(label.score)
        
        
    return (lbs, scr)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [6]:
len(data)

893

In [7]:
pid = list()
for i in range(0,len(data)):
    pid.append(data[i]['postId'])

In [8]:
items = Counter(pid).keys()
print("No of unique items in the list are:", len(items))

No of unique items in the list are: 675


In [9]:
url = list()
for i in range(0, len(data)):
    url.append(data[i]['imgUrl'])

In [10]:
items = Counter(url).keys()
print("No of unique items in the list are:", len(items))

No of unique items in the list are: 485


### Means there are 675 different posts with 893 images in total. 
### However there are only 485 unique images used throughout all the 675 posts
### This implies multiple posts have same images

In [12]:
for i in range(0, len(data)):
    lbs, scr = detect_labels_uri(url[i])
    data[i]['labels'] = lbs 
    data[i]['score'] = scr
    if i%20 == 0:
        print(i)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880


In [13]:
with open('insta_label_score.json', 'w') as json_file:
    json.dump(data, json_file)

In [14]:
# Closing file
f.close()

In [15]:
f1 = open('insta_label_score.json')

In [16]:
dt = json.load(f1)

In [18]:
dt[0:4]

[{'postUrl': 'https://www.instagram.com/p/Ckvlx4fA7ch/',
  'description': 'Photo by @paoloverzone | Still wrapped like mummies after their move to the new Grand Egyptian Museum in Giza from sites across Egypt, statues of pharaohs and gods surround its director, Maj. Gen. Atef Moftah. An engineer by training, Moftah was appointed in 2016 by Egypt’s president, Abdel Fattah el Sisi, to oversee the monumental work of the new museum. \nRead the full story at the link in bio and follow @paoloverzone for more images and stories.',
  'commentCount': 217,
  'likeCount': 22920,
  'pubDate': '2022-11-09T14:30:11.000Z',
  'likedByViewer': 'False',
  'isSidecar': 'False',
  'type': 'Photo',
  'caption': 'Photo by National Geographic on November 09, 2022. May be an image of sculpture.',
  'profileUrl': 'https://www.instagram.com/natgeo',
  'username': 'natgeo',
  'imgUrl': 'https://www.instagram.com/p/Ckvlx4fA7ch/media/?size=l',
  'postId': 2.967756835e+18,
  'timestamp': 1668011837270,
  'query': '

In [26]:
lbs = list()
scr = list()
likeCount = list()
commentCount = list()
caption = list()
for i in range(0, len(data)):
    lbs.append(dt[i]['labels'])
    scr.append(dt[i]['score'])
    likeCount.append(dt[i]['likeCount'])
    commentCount.append(dt[i]['commentCount'])
    caption.append(dt[i]['caption'])

In [27]:
import pandas as pd

In [29]:
len(lbs)

893

In [30]:
len(scr)

893

In [127]:
df = pd.DataFrame(list(zip(url, lbs, scr, likeCount, commentCount, caption)), columns =['imgUrl','labels', 'score', 'likeCount', 'commentCount', 'caption']) 
df.head()

,imgUrl,labels,score,likeCount,commentCount,caption
0,https://www.instagram.com/p/Ckvlx4fA7ch/media/...,"[Temple, Sculpture, Statue, World, Art, Landma...","[0.8743597865104675, 0.8710590600967407, 0.865...",22920,217,"Photo by National Geographic on November 09, 2..."
1,https://www.instagram.com/p/Ckt9EF4AgD5/media/...,"[World, Font, Bedrock, Landscape, Natural land...","[0.898686408996582, 0.8260526657104492, 0.7738...",156864,801,"Photo by National Geographic on November 08, 2..."
2,https://www.instagram.com/p/CktA-_-Ae-l/media/...,"[Sky, Atmosphere, Building, Cloud, Skyscraper,...","[0.9685000777244568, 0.9481807947158813, 0.927...",135166,673,"Photo by National Geographic on November 08, 2..."
3,https://www.instagram.com/p/CkqtXRvApPf/media/...,"[Event, Flash photography, Darkness, Working a...","[0.7158313989639282, 0.693068265914917, 0.6923...",96861,1239,"Photo by National Geographic on November 07, 2..."
4,https://www.instagram.com/p/CkqtXRvApPf/media/...,"[Event, Flash photography, Darkness, Working a...","[0.7158313989639282, 0.693068265914917, 0.6923...",96861,1239,"Photo by National Geographic on November 07, 2..."


In [33]:
df.to_csv('img_lbs_scr.csv')

In [128]:
df['normalizedLikeCount'] = df['likeCount']/(df['likeCount'].max())
df['normalizedCommentCount'] = df['commentCount']/(df['commentCount'].max())
df['engagementScore'] =  .4*df['normalizedLikeCount'] + .6*df['normalizedCommentCount']
df['engagement'] = df.apply(lambda x: 1 if x['engagementScore'] > df['engagementScore'].median() else 0, axis = 1)
df.head()

,imgUrl,labels,score,likeCount,commentCount,caption,normalizedLikeCount,normalizedCommentCount,engagementScore,engagement
0,https://www.instagram.com/p/Ckvlx4fA7ch/media/...,"[Temple, Sculpture, Statue, World, Art, Landma...","[0.8743597865104675, 0.8710590600967407, 0.865...",22920,217,"Photo by National Geographic on November 09, 2...",0.023917,0.004502,0.012268,0
1,https://www.instagram.com/p/Ckt9EF4AgD5/media/...,"[World, Font, Bedrock, Landscape, Natural land...","[0.898686408996582, 0.8260526657104492, 0.7738...",156864,801,"Photo by National Geographic on November 08, 2...",0.163685,0.016619,0.075445,1
2,https://www.instagram.com/p/CktA-_-Ae-l/media/...,"[Sky, Atmosphere, Building, Cloud, Skyscraper,...","[0.9685000777244568, 0.9481807947158813, 0.927...",135166,673,"Photo by National Geographic on November 08, 2...",0.141043,0.013963,0.064795,1
3,https://www.instagram.com/p/CkqtXRvApPf/media/...,"[Event, Flash photography, Darkness, Working a...","[0.7158313989639282, 0.693068265914917, 0.6923...",96861,1239,"Photo by National Geographic on November 07, 2...",0.101073,0.025706,0.055853,0
4,https://www.instagram.com/p/CkqtXRvApPf/media/...,"[Event, Flash photography, Darkness, Working a...","[0.7158313989639282, 0.693068265914917, 0.6923...",96861,1239,"Photo by National Geographic on November 07, 2...",0.101073,0.025706,0.055853,0


In [129]:
df.to_csv('img_lbs_scr_with_engagement.csv')